In [3]:
import gdsfactory as gf
from shapely.geometry.polygon import Polygon
from shapely import affinity
import numpy as np

from gdsfactory.read import cell_from_yaml_template

# from functools import partial

gf.clear_cache()

In [4]:
help(gf.components.rectangle)

Help on function rectangle in module gdsfactory.components.shapes.rectangle:

rectangle(size: 'Size' = (4.0, 2.0), layer: 'LayerSpec' = 'WG', centered: 'bool' = False, port_type: 'str | None' = 'electrical', port_orientations: 'Ints | None' = (180, 90, 0, -90)) -> 'Component'
    Returns a rectangle.

    Args:
        size: (tuple) Width and height of rectangle.
        layer: Specific layer to put polygon geometry on.
        centered: True sets center to (0, 0), False sets south-west to (0, 0).
        port_type: optical, electrical.
        port_orientations: list of port_orientations to add. None adds no ports.



In [15]:
class LargeChips(gf.Component):

    def __init__(self, name, *args, **kwargs):
        super().__init__(name, *args, **kwargs)
        CHIP_X = 9200  # The length of the chip in microns  
        CHIP_Y = 9200  # The width of the chip in microns 
        STEP = 200  # The spacing between the markers in microns                     
        ROW_NUM = 10  # The number of markers in each row
        COL_NUM = 10  # The number of markers in each column
        TEXT_SIZE = 10  # The size of the text in microns
        BOUNDARY = 1  # The width of the boundary in microns
        MARKER_SIZE = 20  # The size of the marker in microns
        self.row_length = CHIP_X
        self.col_length = CHIP_Y
        self.step = STEP
        self.row_num = ROW_NUM
        self.col_num = COL_NUM
        self.text_size = TEXT_SIZE
        self.boundary = BOUNDARY
        self.marker_size = MARKER_SIZE
        self.layer = (8, 0)
        self.layer_screen_marker = (1, 0)
        self.marker_dict = {}
        def add_marker(x, y, layer=(9, 0)):
            self.__create_cross1__(f"{y/1000}", x/1000, x, y, 3, 25, 8, layer=layer)
            self.__create_cross1__(f"{-y/1000}", x/1000, x, -y, 3, 25, 8, layer=layer)
            self.__create_cross1__(f"{y/1000}", -x/1000, -x, y, 3, 25, 8, layer=layer)
            self.__create_cross1__(f"{-y/1000}", -x/1000, -x, -y, 3, 25, 8, layer=layer)
        add_marker(2400, 2400)
        self.create_marker()
        self.create_note("20240920")
        self.create_outline(width=BOUNDARY, mx=0.0)
    
    def __create_align_cross__(self, x, y, layer=None, cross=True):
        """
        The squar marker for EBL alignment
        Args:
            x (_type_): Location of the marker in x
            y (_type_): Location of the marker in y
            layer (_type_, optional): Layers. Defaults to None.
        """
        if layer is None:
            layer = self.layer
        size = 20
        marker = self << gf.components.rectangle(size=[size, size], layer=layer)
        marker.dmove((x-10, y-10))
        
        cross_length = 390
        gap = 100
        
        if cross:
            rect1 = self << gf.components.rectangle(size=[cross_length, size/2], layer=layer)
            rect1.dxmin = marker.dxmax + gap
            rect1.dymin = marker.dymin + size/4
            
            rect2 = self << gf.components.rectangle(size=[cross_length, size/2], layer=layer)
            rect2.dxmax = marker.dxmin - gap
            rect2.dymin = marker.dymin + size/4
            
            rect3 = self << gf.components.rectangle(size=[size/2, cross_length], layer=layer)
            rect3.dxmin = marker.dxmin + size/4
            rect3.dymax = marker.dymax - gap
            
            rect4 = self << gf.components.rectangle(size=[size/2, cross_length], layer=layer)
            rect4.dxmin = marker.dxmin + size/4
            rect4.dymin = marker.dymin + gap
    
    def __create_locate__(self, text, num, x, y, text_size, layer=None):
        if layer is None:
            layer = self.layer
        T1 = self << gf.components.text(text=f"{text},{num}", size=text_size, justify="right",layer=layer)
        T1.dxmin = x
        T1.dymax = y
        
    def __create_cross1__(self, text, num, x, y, cross_width, cross_length, text_size, layer=None):
        if layer is None:
            layer = self.layer
        L1 = self << gf.components.L(cross_width, size=(cross_length, cross_length), layer=layer)
        L2 = self << gf.components.L(cross_width, size=(cross_length, cross_length), layer=layer)
        T1 = self << gf.components.text(text=f"{text}", size=text_size, justify="right",layer=layer)
        N1 = self << gf.components.text(text=f"{num}", size=text_size, justify="left",layer=layer)
        L2.drotate(180,(0, 0))
        L1.dmove((x+cross_width/2, y+cross_width/2))
        L2.dmove((x-cross_width/2, y-cross_width/2))
        T1.dmove((L1.dxmax, L1.dymax-text_size))
        N1.dmove((L2.dxmin-text_size/10, L2.dymin))
        
        
    def create_marker(self, remove_markers=[]):
        row_length, col_length = self.row_length, self.col_length
        # Define the characters for the markers
        characters = [chr(i) for i in range(65, 65 + self.row_num)]  # ASCII 65 = 'A'
        
        # Define the spacing between markers
        dx, dy = self.step, self.step  # 5mm x 5mm assuming units in microns
        self.marker_dx = dx
        self.marker_dy = dy
        marker_dict = {}
        rect = gf.components.rectangle(size=[self.marker_size, self.marker_size], layer=self.layer)
        rect_arr = self.add_ref(rect, spacing=(dx, dy), columns=self.col_num, rows=self.row_num)
        rect_arr.dxmin = - rect_arr.dxsize/2
        rect_arr.dymin = - rect_arr.dysize/2
        
        
        def add_ref_arr(rect):
            defect = 2  # 没数字标记的marker的缺少数  
            rect_arr2 = self.add_ref(rect, spacing=(dx, dy), columns=1, rows=self.row_num-2*defect)
            rect_arr2.dxmin = rect_arr.dxmin - 1*dx
            rect_arr2.dymin = rect_arr.dymin + defect*dy
            
            rect_arr3 = self.add_ref(rect, spacing=(dx, dy), columns=1, rows=self.row_num-2*defect)
            rect_arr3.dxmax = rect_arr.dxmax + 1*dx
            rect_arr3.dymin = rect_arr.dymin + defect*dy
            
            rect_arr4 = self.add_ref(rect, spacing=(dx, dy), columns=self.col_num-2*defect, rows=1)
            rect_arr4.dxmin = rect_arr.dxmin + defect*dx
            rect_arr4.dymin = rect_arr.dymin - 1*dy
            
            rect_arr5 = self.add_ref(rect, spacing=(dx, dy), columns=self.col_num-2*defect, rows=1)
            rect_arr5.dxmin = rect_arr.dxmin + defect*dx
            rect_arr5.dymax = rect_arr.dymax + 1*dy
        
        # add_ref_arr(rect)
        
        def gen_cross(layer, marker_loc=2000, cross=True):
            self.__create_align_cross__(marker_loc, marker_loc, layer, cross=cross)
            self.__create_align_cross__(marker_loc, -marker_loc, layer, cross=cross)
            self.__create_align_cross__(-marker_loc, marker_loc, layer, cross=cross)
            self.__create_align_cross__(-marker_loc, -marker_loc, layer, cross=cross)
        gen_cross(layer=self.layer)
        gen_cross(layer=self.layer, marker_loc=2200, cross=False)
        gen_cross(layer=self.layer, marker_loc=1800, cross=False)
        gen_cross(layer=self.layer, marker_loc=1600, cross=False)

        # Loop through each row and column
        for i, char in enumerate(characters):
            for j in range(0, self.col_num):
                marker = self.__create_locate__(j-int(self.col_num/2)+1, i-int(self.row_num/2)+1, j*dx+self.marker_size+rect_arr.dxmin, i*dy-self.marker_size+rect_arr.dymin, self.text_size)
                index_i = i-int(self.row_num/2)+1
                index_j = j-int(self.col_num/2)+1
                locate_x = j*dx+rect_arr.dxmin
                locate_y = i*dy+rect_arr.dymin
                marker_dict |= {f"{index_j}, {index_i}" : [locate_x+self.marker_size/2, locate_y+self.marker_size/2]}  # +10 means the location of the center
                if (index_i, index_j) not in remove_markers:
                    marker = self.__create_locate__(j-int(self.col_num/2)+1, i-int(self.row_num/2)+1, j*dx+self.marker_size+rect_arr.dxmin, i*dy-self.marker_size+rect_arr.dymin, self.text_size, layer=self.layer_screen_marker)
                    rect = self << gf.components.rectangle(size=[self.marker_size, self.marker_size], layer=self.layer_screen_marker)
                    rect.dxmin, rect.dymin = locate_x, locate_y
        self.marker_dict = marker_dict
        
    def get_position(self, i, j, xshift=0, yshift=0):
        try:
            x, y = self.marker_dict[f"{i}, {j}"]
            return x + xshift, y + yshift
        except:
            print(f"Error: the markers do not create!")
            return None
        
            
    def create_outline(self, width, mx=0, my=0):  # 5mm = 5000 micrometers
        dxmin, dymin = self.dxmin, self.dymin
        dxmax, dymax = self.dxmax, self.dymax
        xgap = self.row_length - (dxmax - dxmin)
        ygap = self.col_length - (dymax - dymin)
        # Create the outer square outline of the marker array
        xshift = dxmin - xgap/2
        yshift = dymin - ygap/2
        # outer_square = self.add_polygon([(0, 0), (0, self.col_length), (self.row_length, self.col_length), (self.row_length, 0)], layer=self.layer_screen_marker)
        c = gf.Component()
        outer_square = c << gf.components.rectangle(size=[self.row_length, self.col_length], layer=self.layer_screen_marker)
        outer_square.dxmin = xshift
        outer_square.dymin = yshift
        
        # Create the inner square
        inner_row_length = self.row_length - 2 * width
        inner_col_length = self.col_length - 2 * width
        # inner_square = self.add_polygon([(width, width), (width, inner_col_length + width), (inner_row_length + width, inner_col_length + width), (inner_row_length + width, width)], layer=self.layer_screen_marker)
        inner_square = c << gf.components.rectangle(size=[inner_row_length, inner_col_length], layer=self.layer_screen_marker)
        inner_square.dxmin = xshift + width
        inner_square.dymin = yshift + width

        # Subtract the inner square from the outer square to get the outline
        square_outline_shape = self << gf.boolean(outer_square, inner_square, operation='xor')
        
        # square_outline_shape_move = affinity.translate(square_outline_shape, xshift+mx, yshift+my)
        # self.add_polygon(square_outline_shape_move, layer=self.layer)
        # self.add_polygon(square_outline_shape_move, layer=self.layer_screen_marker)
        
    def center_of_marker(self, chr1, chr2, chr3, chr4):
        x1, y1 = self.marker_dict[chr1]
        x2, y2 = self.marker_dict[chr2]
        x3, y3 = self.marker_dict[chr3]
        x4, y4 = self.marker_dict[chr4]
        x = (x1 + x2 + x3 + x4) / 4
        y = (y1 + y2 + y3 + y4) / 4
        return x, y
    
    def create_note(self, text):
        note = self << gf.components.text(text=text, position = (0, 0), layer = self.layer, size = 50)
        note.dymin = self.dymax - note.dysize
        note.dxmin = - (note.dxmin + note.dxmax)/2
        note2 = self << gf.components.text(text=text, position = (0, 0), layer = self.layer_screen_marker, size = 50)
        note2.dymin = note.dymin
        note2.dxmin = note.dxmin

In [16]:
gf.clear_cache()
chip = LargeChips("chip8")

ValueError: "LargeChips" object has no field "row_length"

In [36]:
chip.show()

2024-10-25 14:24:47.788 | INFO     | kfactory.kcell:show:8779 - klive v0.3.3: Opened file 'C:\Users\fangr1\AppData\Local\Temp\4254462129.gds'


In [97]:
class LargeChipsSuperconductor(LargeChips):
    def __init__(self, name, *args, **kwargs):
        super().__init__(name, *args, **kwargs)
        self.small_electrode_layer = (2, 0)
        self.big_electrode_layer = (3, 0)
        self.small_electrode_length = 100
        self.small_electrode_width = 2
        # self.create_electrodes()
        self.create_pads()
        self.create_tapes_3(2, 30, 40, self.big_electrode_layer)
        self.create_tapers_1(80, 200, 30, self.big_electrode_layer)
        self.create_tapers_2(30, 80, 20, self.big_electrode_layer)
        self.connect_tapers2_tapers3()

    def create_electrode(self, x, y, length, width, theta, layer):
        electrode = self << gf.components.straight(length=length, width=width, layer=layer)
        electrode.drotate(theta)
        electrode.dxmin = x
        electrode.dymin = y
        return electrode

    def create_electrodes(self):
        gap = 4
        self.small_electrodes = []
        for y in np.linspace(80, 80+3*(gap+self.small_electrode_width), 4):
            small_electrode = self.create_electrode(50, y, self.small_electrode_length, self.small_electrode_width, 0, layer=self.small_electrode_layer)
            self.small_electrodes.append(small_electrode)

    def create_pads(self):
        self.pads = []
        xlocate = 3000
        ylocate = -1750
        ygap = 500
        pad_size = 500
        for x in [-xlocate, xlocate-pad_size]:
            if x == -xlocate:
                y_arr = [ylocate+2*(pad_size+ygap), ylocate+3*(pad_size+ygap)]
            else:
                y_arr = [ylocate, ylocate+pad_size+ygap, ylocate+2*(pad_size+ygap), ylocate+3*(pad_size+ygap)]
            for y in y_arr:
                pad = self << gf.components.rectangle(size=[pad_size, pad_size], layer=self.big_electrode_layer)
                pad.dxmin = x
                pad.dymin = y
                self.pads.append(pad)
    
    def create_tapers_1(self, width1, width2, length, layer):
        self.tapers_1 = []

        taper = self << gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        taper.rotate(180)
        taper.dxmin = self.pads[0].dxmax
        taper.dymin = (self.pads[0].dymin + self.pads[0].dymax) / 2 - width2 / 2
        self.tapers_1.append(taper)

        taper = self << gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        taper.rotate(180)
        taper.dxmin = self.pads[1].dxmax
        taper.dymin = self.pads[1].dymin
        self.tapers_1.append(taper)

        taper = self << gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        taper.dxmax = self.pads[2].dxmin
        taper.dymax = self.pads[2].dymax
        self.tapers_1.append(taper)

        taper = self << gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        taper.dxmax = self.pads[3].dxmin
        taper.dymin = (self.pads[3].dymin + self.pads[3].dymax) / 2 - width2 / 2
        self.tapers_1.append(taper)

        taper = self << gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        taper.dxmax = self.pads[4].dxmin
        taper.dymin = (self.pads[4].dymin + self.pads[4].dymax) / 2 - width2 / 2
        self.tapers_1.append(taper)

        taper = self << gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        taper.dxmax = self.pads[5].dxmin
        taper.dymin = self.pads[5].dymin
        self.tapers_1.append(taper)

    def create_tapers_2(self, width1, width2, length, layer):
        self.tapers_2 = []
        self.tapers_2_component = []
        taper_c = gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        self.tapers_2_component.append(taper_c)
        taper = self << taper_c
        taper.rotate(180)
        x, y = self.get_position(-4, 4, yshift=50)
        taper.dxmin = x
        taper.dymin = y
        self.tapers_2.append(taper)
        taper_c = gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        self.tapers_2_component.append(taper_c)
        taper = self << taper_c
        taper.rotate(180)
        x, y = self.get_position(-4, 3, yshift=50)
        taper.dxmin = x
        taper.dymin = y
        self.tapers_2.append(taper)

    def create_tapes_3(self, width1, width2, length, layer):
        self.tapers_3 = []
        self.tapers_3_component = []

        taper_c = gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        self.tapers_3_component.append(taper_c)
        taper = self << taper_c
        taper.drotate(45)
        # taper.ports["o2"].angle = 135
        x, y = self.get_position(1, 1, xshift=0-width2/2, yshift=100-length/2)
        taper.dxmin = x
        taper.dymin = y
        self.tapers_3.append(taper)

        taper_c = gf.components.taper(length=length, width1=width1, width2=width2, layer=layer)
        self.tapers_3_component.append(taper_c)
        taper = self << taper_c
        taper.drotate(135)
        x, y = self.get_position(0, 1, xshift=0-width2/2, yshift=100-length/2)
        taper.dxmin = x
        taper.dymin = y
        self.tapers_3.append(taper)

    def connect_tapers2_tapers3(self):
        x1, y1 = self.get_position(-1, 4, 100, 0)
        gf.routing.route_single(self, self.tapers_2[0]["o1"], self.tapers_3[0]["o2"], layer=self.big_electrode_layer, route_width=30, end_straight_length=40, radius=20)
        # self.tapers_2_component[0].pprint_ports()


gf.clear_cache()
chip = LargeChipsSuperconductor("chip8")
chip.show()

2024-10-25 15:32:37.000 | WARNING  | gdsfactory.component:__getattribute__:137 - Getting `taper_L30_W80_W200_PNon_50b391c5_0_0.rotate` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `taper_L30_W80_W200_PNon_50b391c5_0_0.drotate` instead. For further information, please consult the migration guide: https://gdsfactory.github.io/gdsfactory/notebooks/21_migration_guide_7_8.html
2024-10-25 15:32:37.003 | WARNING  | gdsfactory.component:__getattribute__:137 - Getting `taper_L30_W80_W200_PNon_50b391c5_0_0.rotate` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `taper_L30_W80_W200_PNon_50b391c5_0_0.drotate` instead. For further information, please consult the migration guide: https://gdsfactory.github.io/gdsfactory/notebooks/21_migration_guide_7_8.html
2024-10-25 15:32:37.016 | WARNING  | gdsfactory.component:__getattribute__:137 - Getting `taper_L20_W30_W80_PNone_91a0092d_0_0.rotate` in um is deprecated and will change to DataBa

C:\Users\fangr1\AppData\Local\Temp\ipykernel_15396\4068858240.py:50: UserWarning: Getting `taper_L30_W80_W200_PNon_50b391c5_0_0.rotate` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `taper_L30_W80_W200_PNon_50b391c5_0_0.drotate` instead.
  taper.rotate(180)
C:\Users\fangr1\AppData\Local\Temp\ipykernel_15396\4068858240.py:56: UserWarning: Getting `taper_L30_W80_W200_PNon_50b391c5_0_0.rotate` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `taper_L30_W80_W200_PNon_50b391c5_0_0.drotate` instead.
  taper.rotate(180)
C:\Users\fangr1\AppData\Local\Temp\ipykernel_15396\4068858240.py:87: UserWarning: Getting `taper_L20_W30_W80_PNone_91a0092d_0_0.rotate` in um is deprecated and will change to DataBaseUnits in gdsfactory9. Please use `taper_L20_W30_W80_PNone_91a0092d_0_0.drotate` instead.
  taper.rotate(180)
C:\Users\fangr1\AppData\Local\Temp\ipykernel_15396\4068858240.py:95: UserWarning: Getting `taper_L20_W30_W80_PNone_91a0092d_

2024-10-25 15:32:37.437 | INFO     | kfactory.kcell:show:8779 - klive v0.3.3: Opened file 'C:\Users\fangr1\AppData\Local\Temp\4068858240.gds'
